### LOGOS detection

In [7]:
from LOGOS.utils.logodetect import logodetection

/Users/vishalsehgal/anaconda3/envs/icevision/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/vishalsehgal/anaconda3/envs/icevision/lib/python3.8/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


INFO     - The mmdet config folder already exists. No need to downloaded it. Path : /Users/vishalsehgal/.icevision/mmdetection_configs/mmdetection_configs-2.20.1/configs | icevision.models.mmdet.download_configs:download_mmdet_configs:17


In [8]:
model = logodetection()

load checkpoint from local path: LOGOS/utils/models/logo_model/logo-retinanet-checkpoint-52k_384_50.pth


/Users/vishalsehgal/anaconda3/envs/icevision/lib/python3.8/site-packages/mmdet/core/anchor/builder.py:16: UserWarning: ``build_anchor_generator`` would be deprecated soon, please use ``build_prior_generator`` 
  warnings.warn(
2023-12-16 17:56:26,406 - mmcv - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}
2023-12-16 17:56:26,407 - mmcv - INFO - load model from: torchvision://resnet50
2023-12-16 17:56:26,407 - mmcv - INFO - load checkpoint from torchvision path: torchvision://resnet50
2023-12-16 17:56:26,503 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

2023-12-16 17:56:26,522 - mmcv - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2023-12-16 17:56:26,551 - mmcv - INFO - initialize RetinaHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01, 'override': {'type': 'Normal', 'name': 'r

load checkpoint from local path: LOGOS/utils/models/logo_model/logo-retinanet-checkpoint-30000_30.pth


2023-12-16 17:56:27,087 - mmcv - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}
2023-12-16 17:56:27,087 - mmcv - INFO - load model from: torchvision://resnet50
2023-12-16 17:56:27,088 - mmcv - INFO - load checkpoint from torchvision path: torchvision://resnet50
2023-12-16 17:56:27,151 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

2023-12-16 17:56:27,167 - mmcv - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2023-12-16 17:56:27,195 - mmcv - INFO - initialize RetinaHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01, 'override': {'type': 'Normal', 'name': 'retina_cls', 'std': 0.01, 'bias_prob': 0.01}}
2023-12-16 17:56:27,252 - mmcv - INFO - 
backbone.conv1.weight - torch.Size([64, 3, 7, 7]): 
PretrainedInit: load from torchvision://resnet50 
 
2023-12-16 17:56:27,253 - mmcv - INFO

In [9]:
def img_replace(page, xref, filename=None, stream=None, pixmap=None):
    if bool(filename) + bool(stream) + bool(pixmap) != 1:
        raise ValueError("Exactly one of filename/stream/pixmap must be given")
    doc = page.parent
    new_xref = page.insert_image(
        page.rect, filename=filename, stream=stream, pixmap=pixmap
    )
    doc.xref_copy(new_xref, xref)
    last_contents_xref = page.get_contents()[-1]
    doc.update_stream(last_contents_xref, b" ")

In [19]:
import fitz
import numpy as np
import cv2

def get_images_from_pdf(path):
    pdf_file = fitz.open(path)
    for page_index in range(len(pdf_file)):
        page = pdf_file[page_index] 
        for _, img in enumerate(page.get_images(), start=1): 
            # get the XREF of the image
            xref = img[0]
            # extract the image bytes
            base_image = pdf_file.extract_image(xref) 
            image_bytes = base_image["image"]
            nparr = np.frombuffer(image_bytes, np.uint8)
            imageFile = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
            bboxes = model.predict(imageFile)
            for box in bboxes:
                x, y, w, h = box
                imageFile[y:y+h, x:x+w, :] = 255
            buff_path = "./static/imgs/img.jpg"
            cv2.imwrite(buff_path, imageFile)
            img_replace(page, xref, filename=buff_path)
    pdf_file.save("./static/removed_logo.pdf")